In [149]:
import pandas as pd
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import evaluate
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report

In [98]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


### Data Process

In [83]:
data = pd.read_csv('./final_data.csv')
data.head()

,singer,song_name,index,emotion,caption,tempo,instruments,tempo(int),tempo(category)
0,.38 Special,Caught Up In You,0,excitement,"catchy and memorable, with a memorable guitar ...",Tempo_120.0,"['Program_-1', 'Program_25', 'Program_62', 'Pr...",120,Tempo_135
1,.38 Special,Fantasy Girl,1,excitement,"The song's melody is catchy and memorable, fea...",Tempo_175.0,"['Program_-1', 'Program_103', 'Program_29', 'P...",175,Tempo_165
2,"10,000 Maniacs",A Campfire Song,2,nostalgia,"The melody is mellow and soothing, with a gent...",Tempo_135.0,"['Program_-1', 'Program_16', 'Program_24', 'Pr...",135,Tempo_135
3,101 Strings,Theme From The Godfather,3,nostalgia,Elegant and sweeping orchestral melody with a ...,Tempo_80.0,"['Program_-1', 'Program_0', 'Program_12', 'Pro...",80,Tempo_75
4,10cc,Dreadlock Holiday,4,excitement,"catchy and upbeat, featuring a memorable guita...",Tempo_103.0,"['Program_-1', 'Program_0']",103,Tempo_105


In [84]:
train_data, valid_data = train_test_split(data, stratify=data['emotion'],test_size= 0.1, random_state=42)

In [85]:
with open("category.pickle","rb") as f:
    data_labels = pickle.load(f)

In [86]:
id2label = {k:l for k, l in enumerate(data_labels)}
label2id = {l:k for k, l in enumerate(data_labels)}
num_labels = len(data_labels)

In [147]:
id2label

{0: 'excitement',
 1: 'nostalgia',
 2: 'love',
 3: 'anger',
 4: 'happiness',
 5: 'sadness',
 6: 'calmness',
 7: 'gratitude',
 8: 'irony',
 9: 'grief',
 10: 'loneliness',
 11: 'anticipation',
 12: 'suspicion',
 13: 'hope',
 14: 'anxiety',
 15: 'lust',
 16: 'determination',
 17: 'Tempo_135',
 18: 'Tempo_165',
 19: 'Tempo_75',
 20: 'Tempo_105',
 21: 'Tempo_45',
 22: 'Tempo_195',
 23: 'Tempo_255',
 24: 'Tempo_225'}

In [141]:
class customDataset():
    def __init__(self, data, tokenizer):
        self.tokenizer = tokenizer
        self.dataset = []
        datas = []
        self.labels = []
        for idx, df in tqdm(data.iterrows()):
            label = [0. for _ in range(num_labels)]
            datas.append(df.caption)
            label[label2id[df.emotion]] = 1.
            label[label2id[df['tempo(category)']]] = 1.
            self.labels.append(label)
        self.dataset =  tokenizer(datas,padding=True, truncation=True,max_length=512 ,return_tensors="pt").to('cuda')
        self.labels= torch.tensor(self.labels)
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.dataset.items()}
        item['labels'] = self.labels[idx].clone().detach()
        return item
        # return {'input_ids' : self.dataset[idx]['input_ids'],
        #         'attention_mask' : self.dataset[idx]['attention_mask'],
        #         'labels' : self.labels[idx]
        #         }

In [89]:
BASE_MODEL = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=num_labels,
           id2label=id2label, label2id=label2id, problem_type = "multi_label_classification").to('cuda')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [142]:
dataset_train = customDataset(train_data, tokenizer =tokenizer)
dataset_valid = customDataset(valid_data, tokenizer =tokenizer)


9126it [00:00, 14266.44it/s]
1014it [00:00, 13912.82it/s]


In [148]:
GLOBAL_SCORE_INDICES = range(0,17)
CAUSE_INDICES = range(17, 25)
def get_preds_from_logits(logits):
    ret = np.zeros(logits.shape)
    
    # The first 5 columns (GLOBAL_SCORE_INDICES) are for global scores. They should be handled with a multiclass approach
    # i.e. we fill 1 to the class with highest probability, and 0 into the other columns
    best_class = np.argmax(logits[:, GLOBAL_SCORE_INDICES], axis=-1)
    ret[list(range(len(ret))), best_class] = 1
    
    # The other columns are for causes and emotions. They should be handled with multilabel approach.
    # i.e. we fill 1 to every class whose score is higher than some threshold
    # In this example, we choose that threshold = 0
    ret[:, CAUSE_INDICES] = (logits[:, CAUSE_INDICES] >= 0).astype(int)
    
    return ret

In [143]:
# clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

# def sigmoid(x):
#    return 1/(1 + np.exp(-x))

# def compute_metrics(eval_pred):

#    predictions, labels = eval_pred
#    predictions = sigmoid(predictions)
#    predictions = (predictions > 0.5).astype(int).reshape(-1)
#    return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))

In [154]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    final_metrics = {}
    
    # Deduce predictions from logits
    predictions = get_preds_from_logits(logits)
    
    # Get f1 metrics for global scoring. Notice that f1_micro = accuracy
    final_metrics["f1_micro_for_global_score"] = f1_score(labels[:, GLOBAL_SCORE_INDICES], predictions[:, GLOBAL_SCORE_INDICES], average="micro")
    final_metrics["f1_macro_for_global_score"] = f1_score(labels[:, GLOBAL_SCORE_INDICES], predictions[:, GLOBAL_SCORE_INDICES], average="macro")
    
    # Get f1 metrics for causes
    final_metrics["f1_micro_for_causes"] = f1_score(labels[:, CAUSE_INDICES], predictions[:, CAUSE_INDICES], average="micro")
    final_metrics["f1_macro_for_causes"] = f1_score(labels[:, CAUSE_INDICES], predictions[:, CAUSE_INDICES], average="macro")
    

    # The global f1_metrics
    final_metrics["f1_micro"] = f1_score(labels, predictions, average="micro")
    final_metrics["f1_macro"] = f1_score(labels, predictions, average="macro")
    
    # Classification report
    # print("Classification report for global scores: ")
    # print(classification_report(labels[:, GLOBAL_SCORE_INDICES], predictions[:, GLOBAL_SCORE_INDICES], zero_division=0))
    # print("Classification report for causes: ")
    # print(classification_report(labels[:, CAUSE_INDICES], predictions[:, CAUSE_INDICES], zero_division=0))
    return final_metrics

In [151]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [153]:
class MultiTaskClassificationTrainer(Trainer):
    def __init__(self, group_weights=None, **kwargs):
        super().__init__(**kwargs)
        self.group_weights = group_weights
        
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0]
        
        global_score_loss = torch.nn.functional.cross_entropy(logits[:, GLOBAL_SCORE_INDICES], labels[:, GLOBAL_SCORE_INDICES])
        cause_loss = torch.nn.functional.binary_cross_entropy_with_logits(logits[:, CAUSE_INDICES], labels[:, CAUSE_INDICES])
        
        loss = self.group_weights[0] * global_score_loss +self.group_weights[1] * cause_loss
        return (loss, outputs) if return_outputs else loss

In [155]:
training_args = TrainingArguments(

   output_dir="my_awesome_model",
   learning_rate=2e-5,
   per_device_train_batch_size=3,
   per_device_eval_batch_size=3,
   num_train_epochs=4,
   weight_decay=0.01,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
)

# trainer = Trainer(

#    model=model,
#    args=training_args,
#    train_dataset=dataset_train,
#    eval_dataset=dataset_valid,
#    tokenizer=tokenizer,
#    data_collator=data_collator,
#    compute_metrics=compute_metrics,
# )

trainer = MultiTaskClassificationTrainer(

   model=model,
   args=training_args,
   train_dataset=dataset_train,
   eval_dataset=dataset_valid,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
   group_weights=(0.7, 4)
)



trainer.train()


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,F1 Micro For Global Score,F1 Macro For Global Score,F1 Micro For Causes,F1 Macro For Causes,F1 Micro,F1 Macro
1,0.925100,3.097902,0.544379,0.147133,0.289504,0.124159,0.426901,0.139781
2,0.989400,3.186205,0.558185,0.156859,0.303233,0.123217,0.438514,0.146093
3,1.375600,2.823989,0.549310,0.152312,0.284553,0.109720,0.427733,0.138683
4,1.080600,3.059764,0.553254,0.150315,0.275626,0.112933,0.424419,0.138353


/home/leesk/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/leesk/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
Checkpoint destination directory my_awesome_model/checkpoint-3042 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/home/leesk/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor pred

TrainOutput(global_step=12168, training_loss=1.1078459283852562, metrics={'train_runtime': 611.6824, 'train_samples_per_second': 59.678, 'train_steps_per_second': 19.893, 'total_flos': 975669234935616.0, 'train_loss': 1.1078459283852562, 'epoch': 4.0})